In [6]:
import pandas as pd
import ast

# Cargar datos de usuarios
df_users = pd.read_csv("australian_user_items.csv")
df_users["items"] = df_users["items"].apply(ast.literal_eval)

# Cargar metadatos de juegos
df_meta = pd.read_csv("steam_juegos_metadata.csv").fillna("")
df_meta["appid"] = df_meta["appid"].astype(str)


In [7]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

# Asegúrate de tener los stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')

stopwords_en = set(stopwords.words("english"))

def limpiar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r"[^\w\s]", "", texto)
    tokens = word_tokenize(texto)
    return [t for t in tokens if t not in stopwords_en and len(t) > 2]

# --- 1. Corpus por usuario ---
corpus_usuarios = []
for row in df_users.itertuples():
    juegos = [j["item_id"] for j in row.items if "item_id" in j]
    if len(juegos) > 1:
        corpus_usuarios.append(juegos)

# --- 2. Corpus por juego con refuerzo de géneros y categorías ---
corpus_metadata = []

for row in df_meta.itertuples():
    tokens = [row.appid]  # El propio ID como token principal
    tokens += limpiar_texto(row.name)
    tokens += limpiar_texto(row.short_description)
    tokens += limpiar_texto(" ".join(row.developers))

    # Reforzar géneros (x3) y categorías (x2)
    if row.genres:
        tokens += limpiar_texto(row.genres) * 3
    if row.categories:
        tokens += limpiar_texto(row.categories) * 2

    corpus_metadata.append(tokens)

# --- 3. Combinar corpus ---
corpus_total = corpus_usuarios + corpus_metadata


[nltk_data] Downloading package punkt to /home/alvaro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/alvaro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
from gensim.models import Word2Vec

modelo_w2v = Word2Vec(
    sentences=corpus_total,
    vector_size=100,     # tamaño del embedding
    window=5,            # tamaño de la ventana de contexto
    min_count=2,         # ignora tokens con menos de 2 apariciones
    sg=1,                # 1 = skip-gram (mejor para recomendaciones)
    workers=4,           # núcleos para paralelizar
    epochs=10            # número de épocas de entrenamiento
)

modelo_w2v.save("modelo_word2vec_mejorado.model")
print("✅ Modelo Word2Vec entrenado y guardado como 'modelo_word2vec_mejorado.model'")


✅ Modelo Word2Vec entrenado y guardado como 'modelo_word2vec_mejorado.model'


In [9]:
df_meta = pd.read_csv("steam_juegos_metadata.csv").fillna("")
id_to_name = dict(zip(df_meta["appid"].astype(str), df_meta["name"]))


In [10]:
def recomendar_similares(juego_id, modelo, id_to_name, topn=20):
    if juego_id not in modelo.wv:
        print(f"⚠️ El juego {juego_id} no está en el modelo.")
        return []

    similares = modelo.wv.most_similar(juego_id, topn=100)
    recomendaciones = []

    for item_id, score in similares:
        if item_id in id_to_name:
            recomendaciones.append((item_id, id_to_name[item_id], round(score, 4)))
        if len(recomendaciones) >= topn:
            break

    return recomendaciones


In [11]:
from gensim.models import Word2Vec

modelo = Word2Vec.load("modelo_word2vec_mejorado.model")
recomendaciones = recomendar_similares("298110", modelo, id_to_name)  # ARK: Survival Evolved

for item_id, nombre, score in recomendaciones:
    print(f"🎮 {item_id} - {nombre} (score: {score})")


🎮 305620 - The Long Dark (score: 0.8529)
🎮 289650 - Assassin's Creed® Unity (score: 0.8438)
🎮 241930 - Middle-earth™: Shadow of Mordor™ (score: 0.8181)
🎮 274940 - Depth (score: 0.8181)
🎮 209660 - Call of Duty®: Advanced Warfare - Gold Edition (score: 0.8137)
🎮 330840 - Game of Thrones - A Telltale Games Series (score: 0.8074)
🎮 323470 - DRAGON BALL XENOVERSE (score: 0.8062)
🎮 331670 - The Jackbox Party Pack (score: 0.7997)
🎮 209650 - Call of Duty®: Advanced Warfare - Gold Edition (score: 0.7899)
🎮 332500 - GRAV (score: 0.7879)
🎮 241560 - The Crew™ (score: 0.7851)
🎮 327440 - Hail to the King: Deathbat (score: 0.7842)
🎮 331500 - Executive Assault (score: 0.782)
🎮 332310 - LEGO® Worlds (score: 0.7746)
🎮 302510 - Ryse: Son of Rome (score: 0.774)
🎮 307690 - Sleeping Dogs: Definitive Edition (score: 0.77)
🎮 330830 - Tales from the Borderlands (score: 0.7622)
🎮 335300 - DARK SOULS™ II: Scholar of the First Sin (score: 0.7576)
🎮 333650 - Pahelika: Revelations (score: 0.7574)
🎮 327860 - Salt (s

In [ ]:
# import pandas as pd
# import requests
# import os
# import json
# import time
# import ast
# from dotenv import load_dotenv

# # === Configuración ===
# load_dotenv()
# API_KEY = os.getenv("STEAM_API_KEY_DIEGO")

# CSV_USUARIOS = "australian_user_items.csv"
# CSV_METADATA = "steam_juegos_metadata.csv"
# PENDIENTES_PATH = "metadata_pendientes.json"

# # === Cargar juegos del dataset de usuarios ===
# print("📥 Cargando juegos desde australian_user_items.csv...")
# df = pd.read_csv(CSV_USUARIOS)
# df["items"] = df["items"].apply(ast.literal_eval)

# juegos_usuarios = set()
# for items in df["items"]:
#     for item in items:
#         juegos_usuarios.add(str(item["item_id"]))

# print(f"🎮 Juegos únicos de usuarios: {len(juegos_usuarios)}")

# # === Cargar juegos ya presentes en metadata ===
# if os.path.exists(CSV_METADATA):
#     df_meta = pd.read_csv(CSV_METADATA)
#     juegos_metadata = set(df_meta["appid"].astype(str))
# else:
#     df_meta = pd.DataFrame(columns=[
#         "appid", "name", "genres", "developers", "short_description",
#         "header_image", "categories", "release_date", "metacritic_score", "price"
#     ])
#     juegos_metadata = set()

# # === Cargar pendientes anteriores (si existen) ===
# if os.path.exists(PENDIENTES_PATH):
#     with open(PENDIENTES_PATH) as f:
#         pendientes = set(json.load(f))
# else:
#     pendientes = set()

# # === Calcular juegos faltantes ===
# faltantes = sorted((juegos_usuarios | pendientes) - juegos_metadata)
# print(f"🧩 Juegos sin metadata: {len(faltantes)}")

# # === Función para obtener metadata desde la API ===
# def obtener_info_juego(appid):
#     url = f"https://store.steampowered.com/api/appdetails?appids={appid}&cc=es&l=english"
#     try:
#         r = requests.get(url, timeout=10)
#         data = r.json()
#         juego_data = data.get(str(appid), {})
#         if not juego_data.get("success"):
#             print(f"❌ AppID {appid} no disponible en Steam.")
#             return "no_existe"
#         info = juego_data["data"]
#         return {
#             "appid": appid,
#             "name": info.get("name", ""),
#             "genres": ", ".join([g["description"] for g in info.get("genres", [])]),
#             "developers": ", ".join(info.get("developers", [])),
#             "short_description": info.get("short_description", ""),
#             "header_image": info.get("header_image", ""),
#             "categories": ", ".join([c["description"] for c in info.get("categories", [])]),
#             "release_date": info.get("release_date", {}).get("date", ""),
#             "metacritic_score": info.get("metacritic", {}).get("score", ""),
#             "price": info.get("price_overview", {}).get("final_formatted", "")
#         }
#     except Exception as e:
#         print(f"⚠️ Error con AppID {appid}: {e}")
#     return None

# # === Bucle de intentos ===
# intentos = 0
# max_intentos = 5

# while faltantes and intentos < max_intentos:
#     intentos += 1
#     print(f"\n🔁 Ronda #{intentos} - Juegos por intentar: {len(faltantes)}")

#     nuevos = []
#     fallidos = []

#     for i, appid in enumerate(faltantes, 1):
#         print(f"🔍 [{i}/{len(faltantes)}] AppID: {appid}...", end=" ")

#         if appid in juegos_metadata:
#             print("✅ Ya estaba.")
#             continue

#         info = obtener_info_juego(appid)
#         if info == "no_existe":
#             print("🚫 Eliminado permanentemente (no existe).")
#             continue  # no se reintentará
#         elif info:
#             nuevos.append(info)
#             print(f"✅ Añadido: {info['name']}")
#         else:
#             fallidos.append(appid)
#             print("❌ Fallido por error temporal.")

#         time.sleep(0.3)  # evitar rate limit

#     # Guardar nuevos
#     if nuevos:
#         nuevos_df = pd.DataFrame(nuevos)
#         df_meta = pd.concat([df_meta, nuevos_df], ignore_index=True)
#         df_meta = df_meta.drop_duplicates(subset="appid")
#         df_meta.to_csv(CSV_METADATA, index=False)
#         juegos_metadata = set(df_meta["appid"].astype(str))
#         print(f"📁 Guardados {len(nuevos)} juegos nuevos.")

#     # Guardar fallidos
#     with open(PENDIENTES_PATH, "w") as f:
#         json.dump(sorted(fallidos), f)

#     # Actualizar para siguiente ronda
#     faltantes = sorted(set(fallidos) - juegos_metadata)

# # === Final ===
# if not faltantes:
#     print("\n✅ Todos los juegos procesados correctamente.")
#     if os.path.exists(PENDIENTES_PATH):
#         os.remove(PENDIENTES_PATH)
# else:
#     print(f"\n⚠️ Finalizado tras {intentos} rondas. Quedan {len(faltantes)} juegos en {PENDIENTES_PATH}")


📥 Cargando juegos desde australian_user_items.csv...
🎮 Juegos únicos de usuarios: 10978
🧩 Juegos sin metadata: 1238

🔁 Ronda #1 - Juegos por intentar: 1238
🔍 [1/1238] AppID: 10000... ⚠️ Error con AppID 10000: 'NoneType' object has no attribute 'get'
❌ Fallido por error temporal.
🔍 [2/1238] AppID: 100410... ⚠️ Error con AppID 100410: 'NoneType' object has no attribute 'get'
❌ Fallido por error temporal.
🔍 [3/1238] AppID: 100970... ⚠️ Error con AppID 100970: 'NoneType' object has no attribute 'get'
❌ Fallido por error temporal.
🔍 [4/1238] AppID: 102700... ⚠️ Error con AppID 102700: 'NoneType' object has no attribute 'get'
❌ Fallido por error temporal.
🔍 [5/1238] AppID: 102820... ⚠️ Error con AppID 102820: 'NoneType' object has no attribute 'get'
❌ Fallido por error temporal.
🔍 [6/1238] AppID: 10400... ⚠️ Error con AppID 10400: 'NoneType' object has no attribute 'get'
❌ Fallido por error temporal.
🔍 [7/1238] AppID: 10410... ⚠️ Error con AppID 10410: 'NoneType' object has no attribute 'get

KeyboardInterrupt: 

In [12]:
import pandas as pd
import ast

# Cargar datos
df = pd.read_csv("australian_user_items.csv")
df["items"] = df["items"].apply(ast.literal_eval)

# Explotar en filas
juegos = []
for row in df.itertuples(index=False):
    for item in row.items:
        juegos.append({
            "user_id": row.user_id,
            "item_id": str(item["item_id"]),
            "item_name": item.get("item_name", "")
        })

df_juegos = pd.DataFrame(juegos)
conteo = df_juegos.groupby("item_id")["user_id"].nunique().reset_index(name="usuarios")

# Filtrar juegos jugados por al menos 50 usuarios
juegos_validos = set(conteo[conteo["usuarios"] >= 50]["item_id"])

# Aplicar filtro al dataset original
df["items"] = df["items"].apply(lambda lista: [i for i in lista if str(i["item_id"]) in juegos_validos])

# Guardar limpio
df.to_csv("australian_user_items_filtrado.csv", index=False)
print(f"✅ Dataset guardado con juegos jugados por ≥ 50 usuarios")


✅ Dataset guardado con juegos jugados por ≥ 50 usuarios


In [13]:
import json

with open("popularidad.json") as f:
    popularidad = json.load(f)

juegos_populares = {k for k, v in popularidad.items() if v >= 0.01}

# Filtro combinado con el anterior
df["items"] = df["items"].apply(lambda lista: [
    i for i in lista if str(i["item_id"]) in juegos_validos and str(i["item_id"]) in juegos_populares
])


In [21]:
import pandas as pd
import ast

# === Configuración ===
ENTRADA = "australian_user_items.csv"
SALIDA = "australian_user_items_filtrado.csv"
METADATA = "steam_juegos_metadata.csv"
MIN_USUARIOS = 50
MIN_METACRITIC = 60  # mínimo de 60 sobre 100

# === Cargar metadatos ===
print("📥 Cargando metadata...")
df_meta = pd.read_csv(METADATA).fillna("")
df_meta["appid"] = df_meta["appid"].astype(str)
df_meta["metacritic_score"] = pd.to_numeric(df_meta["metacritic_score"], errors="coerce").fillna(0)

# Filtrar juegos con puntuación suficiente
juegos_con_buena_nota = set(df_meta[df_meta["metacritic_score"] >= MIN_METACRITIC]["appid"])
print(f"🎮 Juegos con Metacritic ≥ {MIN_METACRITIC}: {len(juegos_con_buena_nota)}")

# === Cargar dataset de usuarios ===
print("📥 Cargando datos de usuarios...")
df = pd.read_csv(ENTRADA)
df["items"] = df["items"].apply(ast.literal_eval)

# === Expandir en filas para contar juegos
juegos = []
for row in df.itertuples(index=False):
    for item in row.items:
        juegos.append({
            "user_id": row.user_id,
            "item_id": str(item["item_id"]),
            "item_name": item.get("item_name", "")
        })

df_juegos = pd.DataFrame(juegos)

# === Filtrar juegos jugados por al menos MIN_USUARIOS
conteo = df_juegos.groupby("item_id")["user_id"].nunique().reset_index(name="usuarios")
juegos_masivos = set(conteo[conteo["usuarios"] >= MIN_USUARIOS]["item_id"])
print(f"✅ Juegos jugados por al menos {MIN_USUARIOS} usuarios: {len(juegos_masivos)}")

# === Intersección final
juegos_validos = juegos_con_buena_nota & juegos_masivos
print(f"🎯 Juegos válidos tras filtros combinados: {len(juegos_validos)}")

# === Aplicar filtro al campo "items"
df["items"] = df["items"].apply(lambda lista: [
    i for i in lista if str(i["item_id"]) in juegos_validos
])

# === Eliminar usuarios vacíos
df = df[df["items"].map(len) > 0].reset_index(drop=True)
print(f"👤 Usuarios restantes tras filtrar: {len(df)}")

# === Guardar resultado
df.to_csv(SALIDA, index=False)
print(f"✅ Dataset limpio guardado en '{SALIDA}'")


📥 Cargando metadata...
🎮 Juegos con Metacritic ≥ 60: 2007
📥 Cargando datos de usuarios...
✅ Juegos jugados por al menos 50 usuarios: 5219
🎯 Juegos válidos tras filtros combinados: 1620
👤 Usuarios restantes tras filtrar: 67237
✅ Dataset limpio guardado en 'australian_user_items_filtrado.csv'


In [23]:
import pandas as pd
import ast
import json
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from gensim.models import Word2Vec

# === Descargas necesarias ===
nltk.download('punkt')
nltk.download('stopwords')
stopwords_en = set(stopwords.words("english"))

# === Archivos ===
CSV_USUARIOS = "australian_user_items_filtrado.csv"
CSV_METADATA = "steam_juegos_metadata.csv"
MODELO_SALIDA = "modelo_word2vec_mejorado.model"

# === Limpieza de texto ===
def limpiar_texto(texto):
    if not isinstance(texto, str):
        return []
    texto = texto.lower()
    texto = re.sub(r"[^\w\s]", "", texto)
    tokens = word_tokenize(texto)
    return [t for t in tokens if t not in stopwords_en and len(t) > 2]

# === 1. Corpus por usuario (comportamiento ponderado) ===
print("📥 Cargando datos de usuarios...")
df_users = pd.read_csv(CSV_USUARIOS)
df_users["items"] = df_users["items"].apply(ast.literal_eval)

corpus_usuarios = []
for row in df_users.itertuples():
    juegos = []
    for j in row.items:
        item_id = str(j["item_id"])
        playtime = float(j.get("playtime_forever", 0))
        repeticiones = min(int(playtime // 60), 10)  # 1 repetición por hora, máximo 10
        if repeticiones > 0:
            juegos += [item_id] * repeticiones
    if len(juegos) > 1:
        corpus_usuarios.append(juegos)

print(f"👤 Corpus de usuarios (ponderado): {len(corpus_usuarios)} secuencias")

# === 2. Corpus por juego (metadata textual) ===
print("📥 Cargando metadata...")
df_meta = pd.read_csv(CSV_METADATA).fillna("")
df_meta["appid"] = df_meta["appid"].astype(str)

# === Filtrar juegos con buena puntuación
df_meta["metacritic_score"] = pd.to_numeric(df_meta["metacritic_score"], errors="coerce").fillna(0)
df_meta = df_meta[df_meta["metacritic_score"] >= 60].reset_index(drop=True)
print(f"🎯 Juegos con metacritic ≥ 60: {len(df_meta)}")

corpus_metadata = []
for row in df_meta.itertuples():
    tokens = [row.appid]
    tokens += limpiar_texto(row.name)
    tokens += limpiar_texto(row.short_description)
    tokens += limpiar_texto(" ".join(row.developers))

    if row.genres:
        tokens += limpiar_texto(row.genres) * 3
    if row.categories:
        tokens += limpiar_texto(row.categories) * 2

    corpus_metadata.append(tokens)

print(f"🎮 Corpus de metadata: {len(corpus_metadata)} juegos")

# === 3. Combinar corpus ===
corpus_total = corpus_usuarios + corpus_metadata
print(f"🧠 Corpus total combinado: {len(corpus_total)} secuencias")

# === 4. Entrenar modelo Word2Vec ===
modelo = Word2Vec(
    sentences=corpus_total,
    vector_size=100,
    window=5,
    min_count=1,
    sg=1,  # skip-gram
    epochs=20,
    workers=4
)

modelo.save(MODELO_SALIDA)
print(f"✅ Modelo Word2Vec combinado guardado como '{MODELO_SALIDA}'")


[nltk_data] Downloading package punkt to /home/alvaro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/alvaro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


📥 Cargando datos de usuarios...
👤 Corpus de usuarios (ponderado): 64184 secuencias
📥 Cargando metadata...
🎯 Juegos con metacritic ≥ 60: 2007
🎮 Corpus de metadata: 2007 juegos
🧠 Corpus total combinado: 66191 secuencias
✅ Modelo Word2Vec combinado guardado como 'modelo_word2vec_mejorado.model'
